<a href="https://colab.research.google.com/github/oavilex/DS_C2_SC2_OscarAvila/blob/main/DS_C2_SC2_OscarAvila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [💽]INSTALEMOS lo requerido

Ya que crearemos un excel de salida requeriremos:

In [92]:
!pip install xlsxwriter

# [♻️] Purgar directorio de trabajo

Ejecutar como paso 0 para eliminar todos los archivos que se descargaran o generaran durante los pasos posteriores

In [94]:
import os
import shutil

def delete_recursive(path):
    if os.path.exists(path):
        if os.path.isfile(path) or os.path.islink(path):
            os.remove(path)
        elif os.path.isdir(path):
            shutil.rmtree(path)
        print(f"[♻️] Eliminado: {path}")
    else:
        print(f"[🔴] La ruta no existe: {path}")


download_folder = os.path.expanduser("sample_data")
workspace = os.path.join(download_folder, "covid-world-vaccination-progress")

delete_recursive(workspace)


[♻️] Eliminado: sample_data/covid-world-vaccination-progress


# [🌎] Descarga fuente de informacion

Ejecutar solo si se desea descargar la fuente de informacion

In [95]:
import os
import requests
import zipfile



url = "https://www.kaggle.com/api/v1/datasets/download/gpreda/covid-world-vaccination-progress"

download_folder = os.path.expanduser("sample_data")
zip_file_path = os.path.join(download_folder, "covid-world-vaccination-progress.zip")
workspace = os.path.join(download_folder, "covid-world-vaccination-progress")

os.makedirs(download_folder, exist_ok=True)
os.makedirs(workspace, exist_ok=True)

print("[🌎] Descargando archivo...")
try:
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(zip_file_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            f.write(chunk)
    print(f"  [🟢] Archivo descargado correctamente en: {zip_file_path}")
except requests.exceptions.RequestException as e:
    print(f"  [🔴]Error al descargar el archivo: {e}")
    exit()

print("[📦] Desempaquetando archivo...")
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(workspace)
    print(f"  [🟢] Archivos desempaquetados en: {workspace}")
except Exception as e:
    print(f"  [🔴]Error  extrayendo el archivo ZIP: {e}")
    exit()

print("[🔥] Eliminando archivo ZIP...")
try:
    os.remove(zip_file_path)
    print(f"  [🟢] Archivo ZIP eliminado: {zip_file_path}")
except Exception as e:
    print(f"  [🔴] Error al eliminar el archivo ZIP: {e}")
    exit()


[🌎] Descargando archivo...
  [🟢] Archivo descargado correctamente en: sample_data/covid-world-vaccination-progress.zip
[📦] Desempaquetando archivo...
  [🟢] Archivos desempaquetados en: sample_data/covid-world-vaccination-progress
[🔥] Eliminando archivo ZIP...
  [🟢] Archivo ZIP eliminado: sample_data/covid-world-vaccination-progress.zip


# [🧑‍🔬] Analisis de informacion

-Recuerda descargar previamente la informacion con el paso de *[🌎] Descarga fuente de informacion*



In [96]:
import os
import pandas as pd
import numpy as np
from tabulate import tabulate
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)


# ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
# ░░░░░░░░░ VARIABLES ░░░░░░░░░
# ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
download_folder = os.path.expanduser("sample_data")
workspace = os.path.join(download_folder, "covid-world-vaccination-progress")
studyObjectFile = os.path.join(workspace, "country_vaccinations.csv")

try:
  # a. Extraer la información del archivo.
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[a.  ][🐋] Lectura de la informacion")
  df = pd.read_csv(studyObjectFile)
  print("[🐋] HECHO!!!")
  # b. Mostrar la estructura y tipos de datos de cada columna para identificar
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[b.  ][🔍] Estructura del DataFrame")
  print(type(df))
  print(df.dtypes)#<- Muestro todas las columnas y los tipos que trae por default...
  print(tabulate(df.head(5), headers='keys', tablefmt='fancy_grid'))#<- aqui analizo una muesta de la informacion presentado en una bonita tabla
                                                              #para ver que puedo hacer con la informacion

  # b.1 Convertir los campos de utilidad
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[b.1][ 🪄] Formateo de campo de fecha a datetime")
  df['date'] = pd.to_datetime(df['date'])

  df['total_vaccinations'] = df['total_vaccinations'].fillna(0)#<- formateo el campo a operar
  df['total_vaccinations'] = df['total_vaccinations'].astype(int)# <- me aseguro de que sean int

  df['daily_vaccinations'] = df['daily_vaccinations'].fillna(0)#<- formateo el campo a operar
  df['daily_vaccinations'] = df['daily_vaccinations'].astype(int)# <- me aseguro de que sean int

  df['daily_vaccinations_raw'] = df['daily_vaccinations_raw'].fillna(0)#<- formateo el campo a operar
  df['daily_vaccinations_raw'] = df['daily_vaccinations_raw'].astype(int)# <- me aseguro de que sean int

  print(df.dtypes)
  selected_columns = ['country', 'date', 'total_vaccinations', 'daily_vaccinations']#<-selecciono solo unas columnas
  df_selected = df[selected_columns] #<- creo una copia con las columnas seleccionadas
  print(f"Esta es solo una muestra de las columnas {selected_columns}")
  print(tabulate(df_selected.head(5),headers='keys', tablefmt='fancy_grid'))#<- aqui despues de la convercion las fecha ya se ve con el formato YYYY-MM-DD HH:MM:SS
  print("...imprimi solo los 5 primeros del datafframe")

  # c. Cantidad de vacunas aplicadas por compañía
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[c  ][🫟] Realizo una operacion de 🐼 para agrupar por [vaccines] y sumarizar el [total_vaccinations] y cuantas 💉 se aplicaron")
  vaccines_summary = df.groupby('vaccines')['total_vaccinations'].sum().reset_index()
  print(tabulate(vaccines_summary.head(5), headers='keys', tablefmt='fancy_grid'))
  print("...imprimi solo los 5 primeros del datafframe")

  # d. Total de vacunas aplicadas en todo el mundo
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[d  ][🫟] Realizo una operacion de 🐼 para agrupar todo y ver cuantas 💉 se aplicaron en todo el 🌎")
  total_vaccinations_worldwide = df['total_vaccinations'].sum()
  print(f"Total de 💉 aplicadas en todo el 🌎:[{total_vaccinations_worldwide}] ")

  # e. Promedio de vacunas aplicadas por país
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[e  ][🫟] Realizo una operacion de 🐼 para agrupar por [country] y sumarizar el [total_vaccinations] y cuantas 💉 se aplicaron por pais")
  average_vaccinations_per_country = df.groupby('country')['total_vaccinations'].mean().reset_index()
  print(tabulate(average_vaccinations_per_country.head(5), headers='keys', tablefmt='fancy_grid'))
  print("...imprimi solo los 5 primeros del datafframe")

  # f. Vacunas aplicadas el 29/01/2021 en todo el mundo
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[f  ][🫟] Realizo una operacion de 🐼 seleccionar el 📆dia:[29/01/2021] y sumar cuantas 💉 se aplicaron")
  per_date = df.groupby('date')['total_vaccinations'].sum().reset_index()
  search= pd.to_datetime('2021-01-29')
  total_per_day = per_date[per_date['date'] == search]['total_vaccinations'].values[0]
  print(f"Total de 💉 aplicadas en todo el 🌎:[{total_per_day}] en el dia:[29/01/2021]")

  # g. Crear conDiferencias con la diferencia entre daily_vaccinations y daily_vaccinations_raw
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[g  ][🧮] Creo una copia del dataset oroginal pero con una columna [diferencias] que contiene la diferencia entre la columna [daily_vaccinations]y[daily_vaccinations_raw]")
  conDiferencias = df.copy()
  conDiferencias['diferencias'] = conDiferencias['daily_vaccinations'] - conDiferencias['daily_vaccinations_raw']
  selected_columns = ['country', 'date', 'daily_vaccinations', 'daily_vaccinations_raw',"diferencias"]#<-selecciono solo unas columnas
  conDiferencias_selected = conDiferencias[selected_columns]
  print(tabulate(conDiferencias_selected.head(5), headers='keys', tablefmt='fancy_grid'))

  # h. Obtener el periodo de tiempo entre la fecha más reciente y la más antigua
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[h  ][🧮] Calculo el rango de dias de la muestra")
  min_date = df['date'].min()
  max_date = df['date'].max()
  time_period = max_date - min_date
  print(f"El periodo de tiempo entre la fecha más reciente y la más antigua es: {time_period}")

  #i. Crear un dataframe nuevo denominado conCantidad que contenga los datos originales
  #y una columna derivada (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna derivada
  # (canVac) con la cantidad de vacunas utilizadas cada día (usar la columna vaccines y separar por el carácter ,
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[i  ][🧮] Creo una copia del dataset oroginal pero con una columna [canVac] que contiene el total de elementos que se encuentran en la columna string [vaccines] que esta separada por comas")
  conCantidad = df.copy()
  conCantidad['canVac'] = conCantidad['vaccines'].str.count(',') + 1
  conCantidad['canVac'] = conCantidad['canVac'].fillna(0).astype(int)
  selected_columns = ['vaccines', 'canVac']#<-selecciono solo unas columnas
  conCantidad_selected = conCantidad[selected_columns]
  print(tabulate(conCantidad_selected.head(5), headers='keys', tablefmt='fancy_grid'))


  # j. Filtrar registros antes del 20 de diciembre de 2020
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[j  ][🧮] Filtrar los registros anteriores al 20 de dic de 2020")
  print(f" Total de la muestra original [{len(df)}] registros")
  antes20 = df[df['date'] < '2020-12-20'].copy()
  print(f" Total de la muestra filtrada [{len(antes20)}] registros")
  selected_columns = ['country', 'date', 'daily_vaccinations']
  antes20_selected = antes20[selected_columns]
  print(tabulate(antes20_selected.head(5), headers='keys', tablefmt='fancy_grid'))


  # k. Filtrar registros donde se haya usado la vacuna Pfizer
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[j  ][🧮] Filtrar los registros anteriores donde se haya usado la vacuna Pfizer")
  print(f" Total de la muestra original [{len(df)}] registros")
  pfizer = df[df['vaccines'].str.contains('Pfizer', na=False)].copy()
  print(f" Total de la muestra filtrada [{len(pfizer)}] registros")
  selected_columns = ['vaccines', 'date', 'daily_vaccinations']
  pfizer_selected = pfizer[selected_columns]
  print(tabulate(pfizer_selected.head(5), headers='keys', tablefmt='fancy_grid'))

  # l. Guarda archivos de resultados
  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░")
  print("[j  ][💾] Salvando los resultados")
  print("[🕛]Esto puede tomar un tiempo...🦥🦥🦥🦥🦥🦥🦥🦥🦥🦥🦥🦥🦥")
  resultados_folder = os.path.join(workspace, "resultados")
  os.makedirs(resultados_folder, exist_ok=True)
  excel_file_path = os.path.join(resultados_folder, "resultadosReto.xlsx")
  with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    conDiferencias.to_excel(writer, sheet_name='conDiferencias', index=False)
    conCantidad.to_excel(writer, sheet_name='conCantidad', index=False)
    antes20.to_excel(writer, sheet_name='antes20', index=False)
    pfizer.to_excel(writer, sheet_name='pfizer', index=False)

  print(f"[📂] Archivo guardado en:[{excel_file_path}]")

  print("░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░END")
except Exception as e:
  print(f"[🔴]Error al procesar:[{e}]")

░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
[a.  ][🐋] Lectura de la informacion
[🐋] HECHO!!!
░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░
[b.  ][🔍] Estructura del DataFrame
<class 'pandas.core.frame.DataFrame'>
country                                 object
iso_code                                object
date                                    object
total_vaccinations                     float64
people_vaccinated                      float64
people_fully_vaccinated                float64
daily_vaccinations_raw                 float64
daily_vaccinations                     float64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
daily_vaccinations_per_million         float64
vaccines                                object
source_name                             object
source_website                          object
dtype: object
╒